In [ ]:
# default_exp core

# core

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [2]:
#export
from data_tool.imports import *

In [3]:
#export
def kwargs2str(**kwargs):
    str_list = [f"{k}: {v}" for k, v in kwargs.items()]
    return ", ".join(str_list)